# PHÂN LOẠI SẮC THÁI COMMENT TRONG TIẾNG VIỆT
---
Phân tích comment của khách hàng về sản phẩm thành 2 sắc thái:

1.   Positive
2.   Negative

Giải pháp của mình tập trung vào data hơn mô hình. Với bài toán này, mình tập trung tiền xử lý dữ liệu, loại bỏ nhiễu, gán nhãn lại các mislabel data. Lý do tập trung vào data hơn vì mình quan sát dữ liệu thấy có khá nhiều nhiễu, gán nhãn sai và lấy từ các trang thương mại điện tử nên từ ngữ lộn xộn, thường không theo văn phong chuẩn mực, cần phải có bước chuẩn hóa. Mô hình mình sử dụng là SVM và feature quen thuộc TF-IDF (5-gram). Lý do sử dụng SVM vì mình thấy SVM khá phù hợp với các bài toán có ít dữ liệu nhưng nhiều features. Mô hình này vẫn cho kết quả khá tốt, thời gian train/predict khá nhanh (train dưới 1 phút với macbook 2015 của mình). Cuối cùng là giải thích về việc dùng Error Analysis để gán lại các Mislabel data.

[LINK VÀ BẢN QUYỀN BÀI VIẾT](https://github.com/swordmanager/sentiment_analysis_nal)

In [ ]:
# -*- coding: utf-8 -*-
from __future__ import print_function
from sklearn import metrics
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer,TfidfTransformer
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
# from sklearn.neural_network import MLPClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import MultinomialNB
# from sklearn.linear_model import SGDClassifier
# from sklearn.neighbors import KNeighborsClassifier
# from sklearn.ensemble import AdaBoostClassifier
# from sklearn.ensemble import RandomForestClassifier
import pandas as pd
#from pyvi import ViTokenizer
import re
import string
import codecs

In [ ]:
#Setup thư viện xử lý tiếng việt
!pip install pyvi

     |████████████████████████████████| 8.5MB 2.5MB/s 
     |████████████████████████████████| 747kB 34.4MB/s 


In [ ]:
from pyvi import ViTokenizer

In [ ]:
#Thực hiện mount tới drive chứa file dữ liệu của Project
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## 1) Xây dựng các Hàm tiền xử lý dữ liệu tiếng việt:
---
* Dữ liệu bình luận (văn nói) nên người dùng thường không quan tâm đến chữ hoa thường khi gõ, đưa hết về lower case.
* Loại bỏ những ký tự kéo dài: Ví dụ: Áo đẹp quáaaaaaaa--> Áo đẹp quá.
* Tiếng Việt có 2 cách bỏ dấu nên đưa về 1 chuẩn. Ví dụ, chữ "Hòa" và "Hoà" đều được chấp nhận trong tiếng Việt. Ngoài ra còn một số trường hợp lỗi font chữ cũng cần chuẩn hóa lại. (các trường hợp dính chữ như: "Giao hàngnhanh" xử ý đc sẽ tốt hơn).
* Chuẩn hóa một số sentiment word: "okie"-->"ok", "okey"-->"ok", authentic--> "chuẩn chính hãng",vv...
* Emoj quy về 2 loại: emojs mang ý nghĩa tích cực (positive): '💯','💗' và emojs mang nghĩa tiêu cực (nagative): '👎','😣'.
* Người dùng đánh giá 1,2 sao (*) quy hết về 1star, trên 3 sao quy hết về 5tar.
Loại bỏ dấu câu (puntuations) và các ký tự nhiễu.
* Xử lý vấn đề phủ định, TF-IDF không xử lý được vấn đề phủ định trong bài toán sentiment. Ví dụ: Cái áo này rất đẹp và Cái áo này chẳng đẹp sẽ không khác nhau nhiều khi chọn feature tf-idf, giải pháp của mình là biến chẳng đẹp thành not-positive, hay không tệ thành not-nagative bằng cách dùng từ điển tâm lý và từ điển phủ định. Từ điển tâm lý mình lấy từ VietSentwordnet 1.0 chỉ lấy những từ có score >0.5 bổ sung 1 số sentiment words đặc thù từ tập train.
Augmentation data bằng cách thêm vào các sample của chính tập train nhưng không dấu. (Bình luận không dấu khá phổ biến).
* Ngoài ra, mình bổ sung vào tập train các sample mới lấy từ chính 2 từ điển positive và nagative. Các từ vựng trong từ điển tích cực gán nhãn 0, các từ vựng từ từ điển tiêu cực gán nhãn 1.

In [ ]:
#Từ điển tích cực, tiêu cực, phủ định
path_nag = '/content/drive/My Drive/Colab Notebooks/10Project/Data5_NLP_02/sentiment_dicts/nag.txt'
path_pos = '/content/drive/My Drive/Colab Notebooks/10Project/Data5_NLP_02/sentiment_dicts/pos.txt'
path_not = '/content/drive/My Drive/Colab Notebooks/10Project/Data5_NLP_02/sentiment_dicts/not.txt'
#-------------------------------------------------------------------------------------------------

with codecs.open(path_nag, 'r', encoding='UTF-8') as f:
    nag = f.readlines()
nag_list = [n.replace('\n', '') for n in nag]

with codecs.open(path_pos, 'r', encoding='UTF-8') as f:
    pos = f.readlines()
pos_list = [n.replace('\n', '') for n in pos]
with codecs.open(path_not, 'r', encoding='UTF-8') as f:
    not_ = f.readlines()
not_list = [n.replace('\n', '') for n in not_]


In [ ]:
not_list

['không', 'vô', 'chẳng', 'đếch', 'chưa', 'đéo', 'kém', 'nỏ', 'not']

In [ ]:
#Xây dựng hàm chuẩn hóa dữ liệu tiêng việt
def normalize_text(text):

    #Remove các ký tự kéo dài: vd: đẹppppppp
    text = re.sub(r'([A-Z])\1+', lambda m: m.group(1).upper(), text, flags=re.IGNORECASE)

    # Chuyển thành chữ thường
    text = text.lower()

    #Chuẩn hóa tiếng Việt, xử lý emoj, chuẩn hóa tiếng Anh, thuật ngữ
    replace_list = {
        'òa': 'oà', 'óa': 'oá', 'ỏa': 'oả', 'õa': 'oã', 'ọa': 'oạ', 'òe': 'oè', 'óe': 'oé','ỏe': 'oẻ',
        'õe': 'oẽ', 'ọe': 'oẹ', 'ùy': 'uỳ', 'úy': 'uý', 'ủy': 'uỷ', 'ũy': 'uỹ','ụy': 'uỵ', 'uả': 'ủa',
        'ả': 'ả', 'ố': 'ố', 'u´': 'ố','ỗ': 'ỗ', 'ồ': 'ồ', 'ổ': 'ổ', 'ấ': 'ấ', 'ẫ': 'ẫ', 'ẩ': 'ẩ',
        'ầ': 'ầ', 'ỏ': 'ỏ', 'ề': 'ề','ễ': 'ễ', 'ắ': 'ắ', 'ủ': 'ủ', 'ế': 'ế', 'ở': 'ở', 'ỉ': 'ỉ',
        'ẻ': 'ẻ', 'àk': u' à ','aˋ': 'à', 'iˋ': 'ì', 'ă´': 'ắ','ử': 'ử', 'e˜': 'ẽ', 'y˜': 'ỹ', 'a´': 'á',
        #Quy các icon về 2 loại emoj: Tích cực hoặc tiêu cực
        "👹": "nagative", "👻": "positive", "💃": "positive",'🤙': ' positive ', '👍': ' positive ',
        "💄": "positive", "💎": "positive", "💩": "positive","😕": "nagative", "😱": "nagative", "😸": "positive",
        "😾": "nagative", "🚫": "nagative",  "🤬": "nagative","🧚": "positive", "🧡": "positive",'🐶':' positive ',
        '👎': ' nagative ', '😣': ' nagative ','✨': ' positive ', '❣': ' positive ','☀': ' positive ',
        '♥': ' positive ', '🤩': ' positive ', 'like': ' positive ', '💌': ' positive ',
        '🤣': ' positive ', '🖤': ' positive ', '🤤': ' positive ', ':(': ' nagative ', '😢': ' nagative ',
        '❤': ' positive ', '😍': ' positive ', '😘': ' positive ', '😪': ' nagative ', '😊': ' positive ',
        '?': ' ? ', '😁': ' positive ', '💖': ' positive ', '😟': ' nagative ', '😭': ' nagative ',
        '💯': ' positive ', '💗': ' positive ', '♡': ' positive ', '💜': ' positive ', '🤗': ' positive ',
        '^^': ' positive ', '😨': ' nagative ', '☺': ' positive ', '💋': ' positive ', '👌': ' positive ',
        '😖': ' nagative ', '😀': ' positive ', ':((': ' nagative ', '😡': ' nagative ', '😠': ' nagative ',
        '😒': ' nagative ', '🙂': ' positive ', '😏': ' nagative ', '😝': ' positive ', '😄': ' positive ',
        '😙': ' positive ', '😤': ' nagative ', '😎': ' positive ', '😆': ' positive ', '💚': ' positive ',
        '✌': ' positive ', '💕': ' positive ', '😞': ' nagative ', '😓': ' nagative ', '️🆗️': ' positive ',
        '😉': ' positive ', '😂': ' positive ', ':v': '  positive ', '=))': '  positive ', '😋': ' positive ',
        '💓': ' positive ', '😐': ' nagative ', ':3': ' positive ', '😫': ' nagative ', '😥': ' nagative ',
        '😃': ' positive ', '😬': ' 😬 ', '😌': ' 😌 ', '💛': ' positive ', '🤝': ' positive ', '🎈': ' positive ',
        '😗': ' positive ', '🤔': ' nagative ', '😑': ' nagative ', '🔥': ' nagative ', '🙏': ' nagative ',
        '🆗': ' positive ', '😻': ' positive ', '💙': ' positive ', '💟': ' positive ',
        '😚': ' positive ', '❌': ' nagative ', '👏': ' positive ', ';)': ' positive ', '<3': ' positive ',
        '🌝': ' positive ',  '🌷': ' positive ', '🌸': ' positive ', '🌺': ' positive ',
        '🌼': ' positive ', '🍓': ' positive ', '🐅': ' positive ', '🐾': ' positive ', '👉': ' positive ',
        '💐': ' positive ', '💞': ' positive ', '💥': ' positive ', '💪': ' positive ',
        '💰': ' positive ',  '😇': ' positive ', '😛': ' positive ', '😜': ' positive ',
        '🙃': ' positive ', '🤑': ' positive ', '🤪': ' positive ','☹': ' nagative ',  '💀': ' nagative ',
        '😔': ' nagative ', '😧': ' nagative ', '😩': ' nagative ', '😰': ' nagative ', '😳': ' nagative ',
        '😵': ' nagative ', '😶': ' nagative ', '🙁': ' nagative ',
        #Chuẩn hóa 1 số sentiment words/English words
        ':))': '  positive ', ':)': ' positive ', 'ô kêi': ' ok ', 'okie': ' ok ', ' o kê ': ' ok ',
        'okey': ' ok ', 'ôkê': ' ok ', 'oki': ' ok ', ' oke ':  ' ok ',' okay':' ok ','okê':' ok ',
        ' tks ': u' cám ơn ', 'thks': u' cám ơn ', 'thanks': u' cám ơn ', 'ths': u' cám ơn ', 'thank': u' cám ơn ',
        '⭐': 'star ', '*': 'star ', '🌟': 'star ', '🎉': u' positive ',
        'kg ': u' không ','not': u' không ', u' kg ': u' không ', '"k ': u' không ',' kh ':u' không ','kô':u' không ','hok':u' không ',' kp ': u' không phải ',u' kô ': u' không ', '"ko ': u' không ', u' ko ': u' không ', u' k ': u' không ', 'khong': u' không ', u' hok ': u' không ',
        'he he': ' positive ','hehe': ' positive ','hihi': ' positive ', 'haha': ' positive ', 'hjhj': ' positive ',
        ' lol ': ' nagative ',' cc ': ' nagative ','cute': u' dễ thương ','huhu': ' nagative ', ' vs ': u' với ', 'wa': ' quá ', 'wá': u' quá', 'j': u' gì ', '“': ' ',
        ' sz ': u' cỡ ', 'size': u' cỡ ', u' đx ': u' được ', 'dk': u' được ', 'dc': u' được ', 'đk': u' được ',
        'đc': u' được ','authentic': u' chuẩn chính hãng ',u' aut ': u' chuẩn chính hãng ', u' auth ': u' chuẩn chính hãng ', 'thick': u' positive ', 'store': u' cửa hàng ',
        'shop': u' cửa hàng ', 'sp': u' sản phẩm ', 'gud': u' tốt ','god': u' tốt ','wel done':' tốt ', 'good': u' tốt ', 'gút': u' tốt ',
        'sấu': u' xấu ','gut': u' tốt ', u' tot ': u' tốt ', u' nice ': u' tốt ', 'perfect': 'rất tốt', 'bt': u' bình thường ',
        'time': u' thời gian ', 'qá': u' quá ', u' ship ': u' giao hàng ', u' m ': u' mình ', u' mik ': u' mình ',
        'ể': 'ể', 'product': 'sản phẩm', 'quality': 'chất lượng','chat':' chất ', 'excelent': 'hoàn hảo', 'bad': 'tệ','fresh': ' tươi ','sad': ' tệ ',
        'date': u' hạn sử dụng ', 'hsd': u' hạn sử dụng ','quickly': u' nhanh ', 'quick': u' nhanh ','fast': u' nhanh ','delivery': u' giao hàng ',u' síp ': u' giao hàng ',
        'beautiful': u' đẹp tuyệt vời ', u' tl ': u' trả lời ', u' r ': u' rồi ', u' shopE ': u' cửa hàng ',u' order ': u' đặt hàng ',
        'chất lg': u' chất lượng ',u' sd ': u' sử dụng ',u' dt ': u' điện thoại ',u' nt ': u' nhắn tin ',u' tl ': u' trả lời ',u' sài ': u' xài ',u'bjo':u' bao giờ ',
        'thik': u' thích ',u' sop ': u' cửa hàng ', ' fb ': ' facebook ', ' face ': ' facebook ', ' very ': u' rất ',u'quả ng ':u' quảng  ',
        'dep': u' đẹp ',u' xau ': u' xấu ','delicious': u' ngon ', u'hàg': u' hàng ', u'qủa': u' quả ',
        'iu': u' yêu ','fake': u' giả mạo ', 'trl': 'trả lời', '><': u' positive ',
        ' por ': u' tệ ',' poor ': u' tệ ', 'ib':u' nhắn tin ', 'rep':u' trả lời ',u'fback':' feedback ','fedback':' feedback ',
        #dưới 3* quy về 1*, trên 3* quy về 5*
        '6 sao': ' 5star ','6 star': ' 5star ', '5star': ' 5star ','5 sao': ' 5star ','5sao': ' 5star ',
        'starstarstarstarstar': ' 5star ', '1 sao': ' 1star ', '1sao': ' 1star ','2 sao':' 1star ','2sao':' 1star ',
        '2 starstar':' 1star ','1star': ' 1star ', '0 sao': ' 1star ', '0star': ' 1star ',}

    for k, v in replace_list.items():
        text = text.replace(k, v)

    # Thực hiện chuyển đổi các dấu câu thành space
    translator = str.maketrans(string.punctuation, ' ' * len(string.punctuation))
    text = text.translate(translator)

    text = ViTokenizer.tokenize(text)
    texts = text.split()
    len_text = len(texts)

    texts = [t.replace('_', ' ') for t in texts]
    for i in range(len_text):
        cp_text = texts[i]
        if cp_text in not_list: # Xử lý vấn đề phủ định (VD: áo này chẳng đẹp--> áo này notpos)
            numb_word = 2 if len_text - i - 1 >= 4 else len_text - i - 1

            for j in range(numb_word):
                if texts[i + j + 1] in pos_list:
                    texts[i] = 'notpos'
                    texts[i + j + 1] = ''

                if texts[i + j + 1] in nag_list:
                    texts[i] = 'notnag'
                    texts[i + j + 1] = ''
        else: #Thêm feature cho những sentiment words (áo này đẹp--> áo này đẹp positive)
            if cp_text in pos_list:
                texts.append('positive')
            elif cp_text in nag_list:
                texts.append('nagative')

    text = u' '.join(texts)

    #remove nốt những ký tự thừa thãi
    text = text.replace(u'"', u' ')
    text = text.replace(u'️', u'')
    text = text.replace('🏻','')
    return text

In [ ]:
#Xây dựng Hàm bỏ dấu trong tiếng việt (Chuyển từ tiếng việt có dấu --> không dấu)
VN_CHARS_LOWER = u'ạảãàáâậầấẩẫăắằặẳẵóòọõỏôộổỗồốơờớợởỡéèẻẹẽêếềệểễúùụủũưựữửừứíìịỉĩýỳỷỵỹđð'
VN_CHARS_UPPER = u'ẠẢÃÀÁÂẬẦẤẨẪĂẮẰẶẲẴÓÒỌÕỎÔỘỔỖỒỐƠỜỚỢỞỠÉÈẺẸẼÊẾỀỆỂỄÚÙỤỦŨƯỰỮỬỪỨÍÌỊỈĨÝỲỶỴỸÐĐ'
VN_CHARS = VN_CHARS_LOWER + VN_CHARS_UPPER

def no_marks(s):
    __INTAB = [ch for ch in VN_CHARS]
    __OUTTAB = "a"*17 + "o"*17 + "e"*11 + "u"*11 + "i"*5 + "y"*5 + "d"*2
    __OUTTAB += "A"*17 + "O"*17 + "E"*11 + "U"*11 + "I"*5 + "Y"*5 + "D"*2
    __r = re.compile("|".join(__INTAB))
    __replaces_dict = dict(zip(__INTAB, __OUTTAB))
    result = __r.sub(lambda m: __replaces_dict[m.group(0)], s)
    return result

In [ ]:
#Test các hàm tiền xử lý dữ liệu

text = "Giao hàng cực nhanh, đặt hôm trước trưa hôm sau đã nhận được._Ngoại hình: Bình mới 100%, không cấn móp gì cả, đẹp hơn trong hình nhiều.. _Khả năng giữ lạnh: mua cà phê sữa đã uống xong, còn đá để từ 9h sáng tới 22h đêm mở ra không tan viên đá nào, rất hài lòng. _Khả năng giữ nóng: chưa test :)) Có quai xách rất tiện lợi, dưới đáy lớp cao su đệm nên để xuống bàn rất chắc chắn không trơn, khó ngã. Có đều giữ lạnh, lạnh luôn ra bên ngoài bình😆"
#text = "Áo này thiết kế rất rất rất đẹpppppppppppp 6 sao; nhưng chất liệu thì không thể nào chấp nhận được 1 sao ^.^ !"
#text = "cái áo này không xấu....!"
text1=normalize_text(text)
print('Raw Text          :',text)
print('------------------------------------------------------')
print('Text sau khi xử lý: ', text1)

Raw Text          : Giao hàng cực nhanh, đặt hôm trước trưa hôm sau đã nhận được._Ngoại hình: Bình mới 100%, không cấn móp gì cả, đẹp hơn trong hình nhiều.. _Khả năng giữ lạnh: mua cà phê sữa đã uống xong, còn đá để từ 9h sáng tới 22h đêm mở ra không tan viên đá nào, rất hài lòng. _Khả năng giữ nóng: chưa test :)) Có quai xách rất tiện lợi, dưới đáy lớp cao su đệm nên để xuống bàn rất chắc chắn không trơn, khó ngã. Có đều giữ lạnh, lạnh luôn ra bên ngoài bình😆
------------------------------------------------------
Text sau khi xử lý:  giao hàng cực nhanh đặt hôm trước trưa hôm sau đã nhận được ngoại hình bình mới 100 notnag cấn  gì cả đẹp hơn trong hình nhiều khả năng giữ lạnh mua cà phê sữa đã uống xong còn đá để từ 9h sáng tới 22h đêm mở ra không tan viên đá nào rất hài lòng khả năng giữ nóng chưa test positive có quai xách rất tiện lợi dưới đáy lớp cao su đệm nên để xuống bàn rất chắc chắn notnag trơn  ngã có đều giữ lạnh lạnh luôn ra bên ngoài bình positive positive positive positi

In [ ]:
#Loại bỏ dấu câu
print(no_marks(text1))

giao hang cuc nhanh dat hom truoc trua hom sau da nhan duoc ngoai hinh binh moi 100 notnag can  gi ca dep hon trong hinh nhieu kha nang giu lanh mua ca phe sua da uong xong con da de tu 9h sang toi 22h dem mo ra khong tan vien da nao rat hai long kha nang giu nong chua test positive co quai xach rat tien loi duoi day lop cao su dem nen de xuong ban rat chac chan notnag tron  nga co deu giu lanh lanh luon ra ben ngoai binh positive positive positive positive positive


## 2) Đọc dữ liệu phục vụ cho Huấn luyện và kiểm thử model
---

In [ ]:
#Viết class thực hiện đọc dữ liệu vào DataFrame
class DataSource(object):
    def _load_raw_data(self, filename, is_train=True):
        a = []
        b = []
        regex = 'train_'
        if not is_train:
            regex = 'test_'

        with open(filename, 'r') as file:
            for line in file:
                if regex in line:
                    b.append(a)
                    a = [line]
                elif line != '\n':
                    a.append(line)
        b.append(a)
        return b[1:]

    def _create_row(self, sample, is_train=True):

        d = {}
        d['id'] = sample[0].replace('\n', '')
        review = ""

        if is_train:
            for clause in sample[1:-1]:
                review += clause.replace('\n', ' ')
                review = review.replace('.', ' ')

            d['label'] = int(sample[-1].replace('\n', ' '))
        else:
            for clause in sample[1:]:
                review += clause.replace('\n', ' ')
                review = review.replace('.', ' ')


        d['review'] = review

        return d

    def load_data(self, filename, is_train=True):

        raw_data = self._load_raw_data(filename, is_train)
        lst = []

        for row in raw_data:
            lst.append(self._create_row(row, is_train))

        return lst

#Hàm transform_to_dataset: sẽ thực hiện:
# 1) Thực hiện tiền xử lý đoạn text.(normalize_text)
# 2) Mỗi một text tạo ra một bản mới không dấu (no_marks - Tăng gấp 2 lần số lượng mẫu)
    def transform_to_dataset(self, x_set,y_set):
        X, y = [], []
        for document, topic in zip(list(x_set), list(y_set)):
            document = normalize_text(document)
            X.append(document.strip())
            y.append(topic)
            #Augmentation bằng cách remove dấu tiếng Việt
            X.append(no_marks(document))
            y.append(topic)
        return X, y

In [ ]:
#Đọc dữ liệu Train vào DataFrame
ds = DataSource()
train_data = pd.DataFrame(ds.load_data('/content/drive/My Drive/Colab Notebooks/10Project/Data5_NLP_02/data_clean/train.crash'))

In [ ]:
train_data

,id,label,review
0,train_000000,0,"""Dung dc sp tot cam on shop Đóng gói sản phẩm ..."
1,train_000001,0,""" Chất lượng sản phẩm tuyệt vời Son mịn nhưn..."
2,train_000002,0,""" Chất lượng sản phẩm tuyệt vời nhưng k có hộp..."
3,train_000003,1,""":(( Mình hơi thất vọng 1 chút vì mình đã kỳ v..."
4,train_000004,1,"""Lần trước mình mua áo gió màu hồng rất ok mà ..."
...,...,...,...
16068,train_016082,1,"""Chẳng biết là Shop có biết đọc hay không mua ..."
16069,train_016083,1,"""Cuốn này mỏng Đọc một buổi sáng là hết Thú ..."
16070,train_016084,0,"""Mang êm chân Đẹp """
16071,train_016085,1,"""Tôi đã nhận đc hàng Sau đây là vài lời muốn n..."


In [ ]:
train_data.iloc[6,2]

'"Đã nhận đc hàng rất nhanh mới đặt buổi tối mà trưa mai là có rồi =}} Đóng gói sản phẩm rất đẹp và chắc chắn Shop phục vụ rất tốt" '

## 3) Bổ sung thêm mẫu từ từ điển Nag - Pos
--

In [ ]:
#Thêm mẫu bằng cách lấy trong từ điển Sentiment (nag/pos)
#Bổ sung thêm các từ khóa nag|pos vào tập dữ liệu
new_data = []
for index,row in enumerate(nag_list):
    new_data.append(['pos'+str(index),'0',row])
for index,row in enumerate(nag_list):
    new_data.append(['nag'+str(index),'1',row])

new_data = pd.DataFrame(new_data,columns=list(['id','label','review']))
train_data = train_data.append(new_data, ignore_index=True)
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16601 entries, 0 to 16600
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      16601 non-null  object
 1   label   16601 non-null  object
 2   review  16601 non-null  object
dtypes: object(3)
memory usage: 389.2+ KB


In [ ]:
train_data.describe(include=['O'])

,id,label,review
count,16601,16601,16601
unique,16601,4,16309
top,train_001560,0,thất vọng
freq,1,8690,6


In [ ]:
train_data

,id,label,review
0,train_000000,0,"""Dung dc sp tot cam on shop Đóng gói sản phẩm ..."
1,train_000001,0,""" Chất lượng sản phẩm tuyệt vời Son mịn nhưn..."
2,train_000002,0,""" Chất lượng sản phẩm tuyệt vời nhưng k có hộp..."
3,train_000003,1,""":(( Mình hơi thất vọng 1 chút vì mình đã kỳ v..."
4,train_000004,1,"""Lần trước mình mua áo gió màu hồng rất ok mà ..."
...,...,...,...
16596,nag259,1,lộn
16597,nag260,1,phức tạp
16598,nag261,1,ế ẩm
16599,nag262,1,ế


In [ ]:
train_data.tail(500)

,id,label,review
16101,pos28,0,ghen
16102,pos29,0,hỗn tạp
16103,pos30,0,dơ
16104,pos31,0,liều lĩnh
16105,pos32,0,dơ bẩn
...,...,...,...
16596,nag259,1,lộn
16597,nag260,1,phức tạp
16598,nag261,1,ế ẩm
16599,nag262,1,ế


## 4) Phân tách dữ liệu thành Train - Test (70%-30%) và Transform dữ liệu
---

In [ ]:
#TÁCH TẬP DỮ LIỆU THÀNH 2 PHẦN TRAIN - TEST ĐỂ XÂY DỰNG MODEL
X_train, X_test, y_train, y_test = train_test_split(train_data.review, 
                                                    train_data.label, 
                                                    test_size=0.3,
                                                    random_state=42)

In [ ]:
X_train

14275                 "Hàng đẹp giá rẻ so với chất lượng" 
1432     "Màu được chỉnh và làm Fake cho nên đừng mong ...
12242                         "Mạng chạy lúc đc lúc ko  " 
7614                           "Đẹp lắm  Cảm ơn shop :))" 
6323     "co sóng mà xài k duoc gi het   ai bik cach nà...
                               ...                        
11284    " Shop phục vụ rất tốt mặc dù lúc đầu do sai s...
11964    "Đây L sản phẩm tệ nhất  Hình in  trầy trụa Ch...
5390     " Chất lượng sản phẩm tuyệt vời  So với giá th...
860      "C chủ và nhân viên dễ thương vui vẻ sẽ ủng hộ...
15795    "Cách shop phục vụ rất tuyệt  Hàng đóng...
Name: review, Length: 11620, dtype: object

In [ ]:
y_train

14275    0
1432     0
12242    1
7614     0
6323     1
        ..
11284    0
11964    1
5390     0
860      0
15795    0
Name: label, Length: 11620, dtype: object

In [ ]:
#Thực hiện transform dữ liệu
X_train, y_train = ds.transform_to_dataset(X_train,y_train)
X_test, y_test = ds.transform_to_dataset(X_test, y_test)

In [ ]:
print('Kích thước tập Train:', len(X_train))
print('Kích thước tập Test:', len(X_test))

Kích thước tập Train: 23240
Kích thước tập Test: 9962


In [ ]:
X_train[:10]

['hàng đẹp giá rẻ so với chất lượng positive positive',
 'hang dep gia re so voi chat luong positive positive',
 'màu được chỉnh và làm giả mạo cho nên đừng mong 100 như ảnh nhưng nói chung vẫn xinh xắn đáng yêu vẫn ngầu chất lượng sản phẩm tuyệt vời positive nagative positive positive positive positive',
 'mau duoc chinh va lam gia mao cho nen dung mong 100 nhu anh nhung noi chung van xinh xan dang yeu van ngau chat luong san pham tuyet voi positive nagative positive positive positive positive',
 'mạng chạy lúc được lúc không positive',
 'mang chay luc duoc luc khong positive',
 'đẹp lắm cảm ơn cửa hàng positive positive positive',
 'dep lam cam on cua hang positive positive positive',
 'co sóng mà xài không duoc gi het ai bik cach nào lm cho no mạnh k',
 'co song ma xai khong duoc gi het ai bik cach nao lm cho no manh k']

In [ ]:
y_train[:10]

[0, 0, 0, 0, 1, 1, 0, 0, 1, 1]

In [ ]:
train_data.review[0]

'"Dung dc sp tot cam on shop Đóng gói sản phẩm rất đẹp và chắc chắn Chất lượng sản phẩm tuyệt vời" '

## 5) Áp dụng model trong phân tích sắc thái 
---

In [ ]:
#Try some models
classifiers = [
               MultinomialNB(),
               LinearSVC(fit_intercept = True,multi_class='crammer_singer', C=1),
               DecisionTreeClassifier()
            
        ]


In [ ]:
#THÊM STOPWORD LÀ NHỮNG TỪ KÉM QUAN TRỌNG
stop_ws = (u'rằng',u'thì',u'là',u'mà')

In [ ]:
for classifier in classifiers:
    steps = []
    steps.append(('CountVectorizer', CountVectorizer(ngram_range=(1,5),stop_words=stop_ws,max_df=0.5, min_df=5)))
    steps.append(('tfidf', TfidfTransformer(use_idf=False, sublinear_tf = True,norm='l2',smooth_idf=True)))
    steps.append(('classifier', classifier))
    clf = Pipeline(steps)
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    report1 = metrics.classification_report(y_test, 
                                            y_pred, 
                                            target_names=['Negative','Positive'],
                                            labels=[1,0], 
                                            digits=3)
    print('KẾT QUẢ VỚI MODEL:',classifier)
    print(report1)


/usr/local/lib/python3.6/dist-packages/numpy/lib/arraysetops.py:565: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask &= (ar1 != a)


KẾT QUẢ VỚI MODEL: MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)
              precision    recall  f1-score   support

    Negative      0.839     0.975     0.902      4600
    Positive      0.975     0.839     0.902      5362

   micro avg      0.902     0.902     0.902      9962
   macro avg      0.907     0.907     0.902      9962
weighted avg      0.912     0.902     0.902      9962



/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


KẾT QUẢ VỚI MODEL: LinearSVC(C=1, class_weight=None, dual=True, fit_intercept=True,
          intercept_scaling=1, loss='squared_hinge', max_iter=1000,
          multi_class='crammer_singer', penalty='l2', random_state=None,
          tol=0.0001, verbose=0)
              precision    recall  f1-score   support

    Negative      0.901     0.940     0.920      4600
    Positive      0.947     0.912     0.929      5362

   micro avg      0.925     0.925     0.925      9962
   macro avg      0.924     0.926     0.925      9962
weighted avg      0.926     0.925     0.925      9962

KẾT QUẢ VỚI MODEL: DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=None, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       rando

/usr/local/lib/python3.6/dist-packages/numpy/lib/arraysetops.py:565: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask &= (ar1 != a)


In [ ]:
#X_train

## 6) Sử dụng Error Analysis để gán lại nhãn:
---
Bằng cách train 2 lần: Lần 1 chia tập train/test theo tỉ lệ 7/3 và lần 2 train overfitting, mình phát hiện ra các trường hợp gán nhãn sai và gán lại nhãn, lặp đi lặp lại quá trình này vài chục lần mình đã gán nhãn lại được khá nhiều data. Cách làm của mình dựa trên ý tưởng của Overfitting, nếu đã dạy mô hình tập dữ liệu A rồi test trên chính tập A đó mà mô hình chỉ đạt độ chính xác thấp chứng tỏ dữ liệu chưa phổ quát, quá ít dữ liệu hoặc gán nhãn sai. VD: Train 7/3 đạt 89%, train overfit đạt chỉ 94% thì chứng tỏ có nhiều data gán nhãn sai. Mình gán lại nhãn đến khi độ chính xác khi train overfit đạt khoảng 98% thì dừng lại, lúc này độ chính xác của train 7/3 đạt khoảng 94%. Việc gán lại nhãn, loại bỏ nhiễu với train data là một phần của data science và hoàn toàn hợp lệ. (Tất nhiên là không động chút nào đến test data)

In [ ]:
#TRAIN OVERFITTING/ERRO ANALYSIS
#Sử dụng model có độ chính xác cao nhất, predict cho chính tập dữ liệu Train.
X_train, y_train = ds.transform_to_dataset(train_data.review, train_data.label)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_train)


In [ ]:
report2 = metrics.classification_report(y_train,
                                        y_pred, 
                                        labels=[1,0], digits=3)
print(report2)

/usr/local/lib/python3.6/dist-packages/numpy/lib/arraysetops.py:565: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask &= (ar1 != a)


              precision    recall  f1-score   support

           1      0.999     0.967     0.982     15294
           0      0.972     0.999     0.985     17908

   micro avg      0.984     0.984     0.984     33202
   macro avg      0.985     0.983     0.984     33202
weighted avg      0.984     0.984     0.984     33202



In [ ]:
for id,x, y1, y2 in zip(train_data.id, X_train, y_train, y_pred):
    if y1 != y2:
        print(id,x, y1, y2)

In [ ]:
#ERRO ANALYSIS
for id,x, y1, y2 in zip(train_data.id, X_train, y_train, y_pred):
    if y1 != y2:
        # CHECK EACH WRONG SAMPLE POSSITIVE/NAGATIVE
        if y1!=0:#1:#0:
            print(id,x, y1, y2)

In [ ]:
#CROSS VALIDATION
cross_score = cross_val_score(clf, X_train,y_train, cv=5)

#REPORT
print('DATASET LEN %d'%(len(X_train)))
print('TRAIN 70/30 \n\n',report1)
print('TRAIN OVERFITING\n\n',report2)
print("CROSSVALIDATION 5 FOLDS: %0.4f (+/- %0.4f)" % (cross_score.mean(), cross_score.std() * 2))

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:191: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  score = y_true == y_pred
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:191: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  score = y_true == y_pred
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:191: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  score = y_true == y_pred
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:191: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  score = y_true == y_pred


DATASET LEN 33202
TRAIN 70/30 

               precision    recall  f1-score   support

    Negative      0.855     0.864     0.859      4600
    Positive      0.882     0.875     0.878      5362

   micro avg      0.870     0.870     0.870      9962
   macro avg      0.869     0.869     0.869      9962
weighted avg      0.870     0.870     0.870      9962

TRAIN OVERFITING

               precision    recall  f1-score   support

           1      0.999     0.967     0.982     15294
           0      0.972     0.999     0.985     17908

   micro avg      0.984     0.984     0.984     33202
   macro avg      0.985     0.983     0.984     33202
weighted avg      0.984     0.984     0.984     33202

CROSSVALIDATION 5 FOLDS: 0.0000 (+/- 0.0000)


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:191: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  score = y_true == y_pred


## Sử dụng model huấn luyện để dự đoán sắc thái tập Test (Gán nhãn setiment)
---

In [ ]:
#Đọc dữ liệu Test
test_data = pd.DataFrame(ds.load_data('/content/drive/My Drive/Colab Notebooks/10Project/Data5_NLP_02/data_clean/test.crash', is_train=False))
test_data

,id,review
0,test_000000,"""Chưa dùng thử nên chưa biết"""
1,test_000001,""" Không đáng tiềnVì ngay đợt sale nên mới mua ..."
2,test_000002,"""Cám ơn shop Đóng gói sản phẩm rất đẹp và chắ..."
3,test_000003,"""Vải đẹp phom oki luôn quá ưng"""
4,test_000004,"""Chuẩn hàng đóng gói đẹp"""
...,...,...
10976,test_010976,""" Thời gian giao hàng rất nhanh ngon mà cay qu..."
10977,test_010977,"""Sản phẩm hơi cũ"""
10978,test_010978,"""Sản phẩm chắc chắn nhưng k bóng bằng trong hì..."
10979,test_010979,""" Chất lượng sản phẩm tuyệt vời có mùi thơm rấ..."


In [ ]:
test_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10981 entries, 0 to 5490
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      10981 non-null  object
 1   review  10981 non-null  object
 2   label   10981 non-null  object
dtypes: object(3)
memory usage: 343.2+ KB


In [ ]:
# #SAVE FILE SUBMIT
test_list = []
for document in test_data.review:
    document = normalize_text(document)
    test_list.append(document)
y_predict = clf.predict(test_list)
test_data['label'] = y_predict
test_data['content'] = test_list
test_data = test_data.sort_values(by=['label'])#Sắp xếp lại dữ liệu theo label


In [ ]:
test_data[['id','review','content', 'label']].to_excel('submit.xlsx',
                                                     encoding='utf-8',
                                                     index=False)
test_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10981 entries, 0 to 5490
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   id       10981 non-null  object
 1   review   10981 non-null  object
 2   label    10981 non-null  object
 3   content  10981 non-null  object
dtypes: object(4)
memory usage: 428.9+ KB


In [ ]:
test_data[test_data.label=='1']

,id,review,label,content
6635,test_006635,"""Shop đóng gói k kỹ càng Khi mở từ dưới đáy h...",1,cửa hàng đóng gói không kỹ càng khi mở từ dưới...
10552,test_010552,"""Quần co giãn thực tế không được như hình ảnh ...",1,quần co giãn thực tế notpos như hình ảnh giá cao
8584,test_008584,"""sản phẩm chất lượng tốt nhưng tiền phí vận ch...",1,sản phẩm chất lượng tốt nhưng tiền phí vận chu...
8606,test_008606,"""not bad""",1,notnag
8738,test_008738,"""Sản phẩm kém chất lượng Mang 4kg chưa đc 3 n...",1,sản phẩm notpos mang 4 notpos chưa 3 ngày đã...
...,...,...,...,...
6638,test_006638,""" Chất lượng sản phẩm kém Mới dùng đã hư rồi """,1,chất lượng sản phẩm kém mới dùng đã hư rồi pos...
5748,test_005748,""" Rất không đáng tiền Shop phục vụ rất kémMình...",1,rất không đáng tiền cửa hàng phục vụ rất kémìn...
2416,test_002416,"""có mùi hôi ăn không được dở tệ cũng có khi ă...",1,có mùi hôi ăn không được dở tệ cũng có khi ăn ...
2422,test_002422,"""Hộp bị vỡ """,1,hộp bị vỡ nagative
